In [64]:
%set_env QXToken=ee6a8eeaba4e73fb88220e80cf1ae551b1444e2aea7f6c1c2cc0038aeb24a66f464f96de618f1b057931b21600f3bc71b34b3d37d3258fd215872f4bb93edede

env: QXToken=ee6a8eeaba4e73fb88220e80cf1ae551b1444e2aea7f6c1c2cc0038aeb24a66f464f96de618f1b057931b21600f3bc71b34b3d37d3258fd215872f4bb93edede


In [1]:
from qiskit.circuit.library import EfficientSU2

num_qubits = 50
reps = 2
abstract_circuit = EfficientSU2(num_qubits, reps=reps, entanglement="pairwise")

In [2]:
import numpy as np

num_parameters = abstract_circuit.num_parameters
param_values = np.random.uniform(-np.pi, np.pi, size=num_parameters)

abstract_circuit.assign_parameters(param_values, inplace=True)

In [3]:
from qiskit.circuit.library import UnitaryOverlap

abstract_circuit.barrier()
abstract_circuit = UnitaryOverlap(abstract_circuit, abstract_circuit)

In [4]:
from qiskit.quantum_info import SparsePauliOp

paulis = ["".join("Z" if i == q else "I" for i in range(num_qubits)) for q in range(num_qubits)]
abstract_observables = [SparsePauliOp(pauli) for pauli in paulis]

In [5]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
backend = service.least_busy(min_num_qubits=127)
backend.name

'ibm_osaka'

In [6]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
target_circuit = pm.run(abstract_circuit)

In [7]:
layout = target_circuit.layout
target_observables = [abs_obs.apply_layout(layout=layout) for abs_obs in abstract_observables]

In [74]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator, EstimatorOptions, Batch
from qiskit_ibm_runtime.options import TwirlingOptions, ResilienceOptionsV2

pub = (target_circuit, target_observables)
default_shots = 10_000

primitive_results = []

with Batch(backend=backend) as batch:
    options_ex1 = EstimatorOptions() 
    options_ex1.optimization_level = 0
    options_ex1.resilience_level = 0
    options_ex1.default_shots = default_shots
    
    estimator = Estimator(options=options_ex1)
    job_ex1 = estimator.run(pubs=[pub])
    
    
    options_ex2 = EstimatorOptions()
    options_ex2.default_shots = default_shots
    options_ex2.dynamical_decoupling = DynamicalDecouplingOptions()  
    
    estimator = Estimator(options=options_ex2)
    job_ex2 = estimator.run(pubs=[pub])


    options_ex3 = EstimatorOptions()
    options_ex3.default_shots = default_shots
    
    options_ex3.resilience = ResilienceOptionsV2()
    options_ex3.twirling = TwirlingOptions()
    
    estimator = Estimator(options=options_ex3)
    job_ex3 = estimator.run(pubs=[pub])
    
    primitive_results.append(job_ex1.result())
    primitive_results.append(job_ex2.result())
    primitive_results.append(job_ex3.result())
    
    from qiskit_ibm_runtime.options import ZneOptions
with Batch(backend=backend) as batch:
    # Excercise-4a: Zero Noise Extrapolation (extrapolator="exponential" | noise_factors=(1, 3, 5))
    options_ex4a = EstimatorOptions()
    options_ex4a.default_shots = default_shots
    
    options_ex4a.resilience = ResilienceOptionsV2(
        zne_mitigation=True,
        zne=ZneOptions(
            extrapolator="exponential", 
            noise_factors=[1, 3, 5]
        )
    )

    estimator = Estimator(options=options_ex4a)
    job_ex4a = estimator.run(pubs=[pub])
    
    
    options_ex4b = EstimatorOptions()
    options_ex4b.default_shots = default_shots
    
    options_ex4b.resilience = ResilienceOptionsV2(
        zne_mitigation=True,
        zne=ZneOptions(
            extrapolator="linear", 
            noise_factors=[1, 3, 5]
        )
    )
    
    estimator = Estimator(options=options_ex4b)
    job_ex4b = estimator.run(pubs=[pub])

    
    options_ex5 = EstimatorOptions()
    options_ex5.default_shots = default_shots
    
    options_ex5.resilience = ResilienceOptionsV2()
    options_ex5.twirling = TwirlingOptions()
    
    options_ex5.resilience = ResilienceOptionsV2(
        zne_mitigation=True,
        zne=ZneOptions(
            extrapolator=("exponential", "linear"), 
            noise_factors=[1, 3, 5]
        )
    )
    
    estimator = Estimator(options=options_ex5)
    job_ex5 = estimator.run(pubs=[pub])
    
    primitive_results.append(job_ex4a.result())
    primitive_results.append(job_ex4b.result())
    primitive_results.append(job_ex5.result())
    
    options_ex6 = EstimatorOptions()
options_ex6.default_shots = default_shots

options_ex6.resilience = ResilienceOptionsV2(
    zne_mitigation=True,
    zne=ZneOptions(
        extrapolator=("exponential", "linear"),
        noise_factors=[1, 3, 5]  
    )
)

options_ex6.dynamical_decoupling = DynamicalDecouplingOptions()

options_ex6.twirling = TwirlingOptions()

estimator = Estimator(mode=backend, options=options_ex6)
job_ex6 = estimator.run(pubs=[pub])
primitive_results.append(job_ex6.result())

In [73]:
from qiskit.visualization import plot_bloch_multivector, plot_histogram

print(target_circuit.draw())


global phase: 5π/4
                   ┌─────────────┐      ┌────┐     ┌─────────────┐ ┌────┐»
        q_47 -> 0 ─┤ Rz(-0.1266) ├──────┤ √X ├─────┤ Rz(-1.5224) ├─┤ √X ├»
                   └────┬────┬───┘ ┌────┴────┴───┐ └────┬────┬───┘ └────┘»
        q_48 -> 1 ──────┤ √X ├─────┤ Rz(-1.7547) ├──────┤ √X ├───────────»
                   ┌────┴────┴───┐ └────┬────┬───┘  ┌───┴────┴───┐ ┌────┐»
        q_49 -> 2 ─┤ Rz(0.53387) ├──────┤ √X ├──────┤ Rz(-1.295) ├─┤ √X ├»
                   └─────────────┘      └────┘      └────────────┘ └────┘»
   ancilla_0 -> 3 ───────────────────────────────────────────────────────»
                                                                         »
   ancilla_1 -> 4 ───────────────────────────────────────────────────────»
                                                                         »
   ancilla_2 -> 5 ───────────────────────────────────────────────────────»
                                                                         »
   anc